# medullo-diff
Medulloblastomas were estimated at 18% in previous N.G. paper; however, pedpancan is at 14%.
- Is this statistically different?
- Why the difference?
    - ICGC are the different samples
    - Could be hg19 or could be adult
    
## Conclusion
The difference is driven by 7 tumors previously classified as ecDNA+ MB and now classified as ecDNA-. In 6 of 7 cases, a low-copy cyclic amplicon was previously detected which may alternately represent ecDNA or HSR. Due to differences in our updated methods, these low-copy amplifications are not detected in this analysis. One tumor was previously classified as ecDNA+ based on weak evidence of a cyclic amplification, which probably represents a false positive and is corrected herein.

In [1]:
import pandas as pd

In [39]:
SUPPLEMENTARY_TABLES_PATH="/Users/ochapman/projects/pedpancan_ecdna/data/Supplementary Tables.xlsx"
MEDULLO_TABLES_PATH="/Users/ochapman/projects/medullo-ecdna/data/41588_2023_1551_MOESM4_ESM.xlsx"
def import_biosamples():
    df = pd.read_excel(SUPPLEMENTARY_TABLES_PATH,sheet_name="2. Biosamples",index_col=0)
    # hack hack, these are 2 biosamples of the same tumor at the same timepoint
    df.loc["SJMB009_D","in_unique_tumor_set"] = True
    df.loc["SJMB009_D","in_unique_patient_set"] = True
    df.loc["SJMB009_E","in_unique_tumor_set"] = False
    df.loc["SJMB009_E","in_unique_patient_set"] = False
    return df
def import_medullo_biosamples():
    return pd.read_excel(MEDULLO_TABLES_PATH,sheet_name="2 WGS Sample Cohort",index_col=0)
def import_medullo_patients():
    return pd.read_excel(MEDULLO_TABLES_PATH,sheet_name="1 WGS Patient Cohort",index_col=0)

In [40]:
bs = import_biosamples()
bsm = import_medullo_biosamples()
bs = bs[bs.cancer_type == "MBL"]
bsm = bsm[~bsm.index.isin(bs[~bs.in_unique_patient_set].index)]
bs = bs[bs.in_unique_patient_set]

In [43]:
print(f"samples in pedpancan not in medullo: {len(set(bs.index)-set(bsm.index))}")
# Not really sure why BS_M16CDR44 is not included in the medullo dataset but no consequence.
print(f"samples in medullo not in pedpancan: {len(set(bsm.index)-set(bs.index))}")

samples in pedpancan not in medullo: 1
samples in medullo not in pedpancan: 293


In [64]:
not_in_both = set(bsm.index)-set(bs.index)
in_both = set(bsm.index)&set(bs.index)

In [70]:
# Any different classifications?
bsm_bs = bsm[bsm.index.isin(in_both)].copy()
bsm_bs["medullo_ecDNA"] = bsm_bs.ecDNA > 0
bsm_bs["pedpancan_ecDNA"] = bs.loc[bsm_bs.index,"ecDNA_sequences_detected"] > 0
bsm_bs.drop(["ecDNA","Aliases"],axis=1,inplace=True)
print(pd.crosstab(bsm_bs.medullo_ecDNA, bsm_bs.pedpancan_ecDNA)) # oh dear
bsm_bs[bsm_bs.medullo_ecDNA != bsm_bs.pedpancan_ecDNA]

pedpancan_ecDNA  False  True 
medullo_ecDNA                
False              145      0
True                 6     24


,Patient_ID,Tumor_type,in_pedpancan,medullo_ecDNA,pedpancan_ecDNA
Sample_ID,,,,,
BS_NMQTPVFW,PT_7QKGGMKV,primary,True,True,False
BS_FVYBGMG1,PT_9DMMZ08D,primary,True,True,False
BS_FT95T4YP,PT_NDVJE438,primary,True,True,False
BS_KRYYDC2X,PT_RH59ET3M,primary,True,True,False
SJMB003_D,SJ008003,primary,True,True,False
SJMB030582_D1,SJ030582,primary,True,True,False


In [51]:
bsm["in_pedpancan"] = bsm.index.isin(in_both)

In [54]:
pd.crosstab(bsm.ecDNA > 0, bsm.in_pedpancan)

in_pedpancan,False,True
ecDNA,,
False,240,145
True,53,30


In [72]:
len(bs[(bs.ecDNA_sequences_detected > 0) & bs.in_unique_patient_set])

24

In [73]:
len(bs[bs.in_unique_patient_set])

176

In [77]:
24/(176)

0.13636363636363635

In [78]:
30/(176)

0.17045454545454544

In [79]:
len(in_both)

175